#INICIANDO O AMBIENTE




INSTALANDO AS BIBLIOTECAS 

In [ ]:
pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
!pip install pandera

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 9.4 MB/s eta 0:00:00


In [ ]:
 !pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 15.5 MB/s eta 0:00:00


DECLARAÇÃO DE LIBS

In [ ]:
import os
import numpy as np
import pandas as pd
import pandera as pa
from google.cloud import storage
from datetime import date
from google.colab import drive
from google.cloud import bigquery

import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

CONECTANDO AO AMBIENTE GCP 

In [ ]:
# Acesso ao GCP
serviceAccount = '/content/skilled-tuner-377718-395f300c8996.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

#CURSOS

###RECEBIMENTO DOS DADOS VIA GCP

In [ ]:
"""Lists all the blobs in the bucket."""

storage_client = storage.Client()
bucket_name = storage_client.get_bucket('projetofinal_soulcode')
# Note: Client.list_blobs requires at least package version 1.17.0.
blobs = storage_client.list_blobs(bucket_name)
# Note: The call returns a response only when the iterator is consumed.

brutus = [] #lista que vai receber os dados brutos 
brutes= [] #lista que vai receber o nome dos arquivo para fazer a alteração no campo curso 
nomeDosDf= [] #lista que vai servir como caminho do arquivo bruto para envio

for blob in blobs:
  if(blob.name[0] =='b' and blob.name[-1] =='x' and blob.name != 'brutos/usuarios.xlsx' and blob.name != 'brutos/leaderboard.xlsx'):
    brutus.append(blob.name)
    brutes.append(blob.name)   
    nomeDosDf.append(blob.name)
    print(brutus)
    #o dado que está sendo recebido é o caminho do arquivo(pasta e nome do arquivo) exemplo: brutos/usuarios.xlsx
    #todas as 3 listas estão recebendo o mesmo dado, mas a frente irá ter os tratamento deles e atribuir a suas respectivas finalidades 

**RECEBIMENTO DADOS**<BR>
Fazendo um for para pegar os caminhos dos aququivos e logo em seguida colocar as tabelas dentro de uma lista

In [ ]:
#atribuindo a uma variável o tamanho da lista para usarmos de métrica 

qtdTabela = len(brutus)

In [ ]:
#aplicando o nome de cada elemento da lista no caminho para que todos os DataFrames possam ser puxados de uma vez do bucket 
#e armazenados na lista 'brutus[]'

for i in range(qtdTabela):
  path = f'gs://projetofinal_soulcode/{brutus[i]}'  
  brutus[i]= pd.read_excel(path)

In [ ]:
brutus[0]

###PRÉ ANALISE

**DADOS BRUTOS BACKUP**

In [ ]:
#fazendo um backup da lista brutus[](que vai sofrer alteração) para uma nova lista
#isso foi feito para que possamos subir os arquivos brutos no final da automoção 
dados_cursos_bruto = []
for i in range(qtdTabela):
  dados_cursos_bruto.append(brutus[i].copy())


###TRATAMENTO DE DADOS

**DICIONARIO DE DADOS** 

In [ ]:
#criando um dicionário de dados para ser alterado o valor de brutes[] para o valor do dicionário   
dicionarioDeCursos = { 'brutos/adminbd.xlsx': 'Administração de Banco de Dados',
'brutos/androidfirebase.xlsx': 'Desenvolvimento Android e Firebase',
'brutos/angular.xlsx' : 'Angular',
'brutos/bancodedadosrelacional(1).xlsx': 'Banco de Dados Relacional',
'brutos/bancodedadosrelacional.xlsx': 'Banco de Dados Relacional',
'brutos/crieseuprimeiroappkotlin.xlsx': 'Crie Seu Primeiro App Com Kotlin',
'brutos/dadosfinanceiroscompython.xlsx': 'Análise de Dados Financeiros Com Python',
'brutos/datastudio.xlsx': 'Data Studio',
'brutos/desenvolvimentowebbootstrap.xlsx': 'Desenvolvimento Web com Bootstrap',
'brutos/desenvolvimentoweborientadoaobjeto.xlsx': 'Desenvolvimento Web Orientado a Objetos',
'brutos/devops.xlsx': 'DevOps',
'brutos/gitgithub.xlsx': 'Git e Github',
'brutos/googlecloud.xlsx': 'Google Cloud Platform',
'brutos/implementacaobd.xlsx': 'Implementação de Banco de Dados',
'brutos/infraestruturadedados.xlsx': 'Infraestrutura de Dados',
'brutos/introaws.xlsx': 'Introdução à Amazon Web Services (AWS)',
'brutos/introducaoaia.xlsx': 'Introdução à Inteligência Artificial',
'brutos/introducaodesenvolvimentoweb.xlsx': 'Introdução ao Desenvolvimento Web',
'brutos/javaintroducao.xlsx': 'Java: Introdução à Linguagem',
'brutos/javaorientado.xlsx': 'Java: Orientação a Objetos',
'brutos/javaspring.xlsx': 'Java Spring Boot',
'brutos/javaweb.xlsx': 'Java Web: Aprenda a Criar Sistemas Web Utilizando Java',
'brutos/kotlinbasicoaoavancado.xlsx': 'Kotlin: Básico ao Avançado',
'brutos/logicadeprogramacao.xlsx': 'Lógica de Programação',
'brutos/metodologiaageis.xlsx': 'Metodologias Ágeis',
'brutos/nosql.xlsx': 'Bancos de Dados Não Relacionais (NoSQL)',
'brutos/powerbi.xlsx': 'Power BI',
'brutos/pythonfinance2.xlsx': 'Python e Dados Financeiros',
'brutos/pythonintroducaoalinguagem.xlsx': 'Python: Introdução à Linguagem',
'brutos/pythonmachinelearning.xlsx': 'Machine Learning com Python',
'brutos/pythonmanipulacaodedados.xlsx': 'Python: Manipulação de Dados Com Pandas e PySpark',
'brutos/pythonorientadoaobjeto.xlsx': 'Python: Orientação a Objetos e Manipulação de Dados',
'brutos/salesforce.xlsx': 'Salesforce'}

**CRIAÇÃO DA COLUNA CURSO**

In [ ]:
#indo ao dicionario, recebendo e colocando em brutes o valor(nome das coluna) 
for i in range(qtdTabela):
  for chave, valor in dicionarioDeCursos.items():
      if (brutes[i]==chave):
        brutes[i] = valor
else:('Nãodeu')

In [ ]:
#inserindo a coluna cursos e atribuindo o valor dela respectivamente com a quantidade de cursos 
for i in range(qtdTabela):
  valor = brutes[i]
  brutus[i] = brutus[i].assign(Curso=valor)
  

**TRADUÇÃO DAS COLUNAS**

In [ ]:
#traduzindo todos os nomes de colunas, de todas as tabelas, para renomear para o PT-BR 
for i in range(qtdTabela):
  brutus[i].rename(columns={'ID_usuário': 'ID_usuario',
                   'Número_de_série_certificado': 'Numero_de_serie_certificado',
                   'name': 'Nome',
                   'email': 'E-mail',
                   'user_id': 'ID_usuario',
                   'enrollments_created_at': 'Matriculado_em',
                   'percent_complete': 'Percentual_completado',
                   'certificate_serial_number': 'Numero_de_serie_certificado'
                   },
                   inplace = True)

**ALTERAÇÃO NA DATA/HORA**

In [ ]:
# Converter a coluna de datas para o formato de data/hora se já não tiver (de todos os 33 df)
for i in range(qtdTabela):
  brutus[i]['Matriculado_em'] = pd.to_datetime(brutus[i]['Matriculado_em'])
  brutus[i]['Matriculado_em'] = brutus[i]['Matriculado_em'].dt.strftime('%d-%m-%Y %H:%M')

**ALTERAÇÃO NA COLUNA NOME**

In [ ]:
# Para colocar a primeira letra de cada nome em letra maiúscula(de todos os 33 df): 
for i in range(qtdTabela):
  brutus[i]['Nome'] = brutus[i]['Nome'].str.split().apply(lambda x: ' '.join([word.title() for word in x]))

**NULOS OU AUSENTES**

In [ ]:
# Verificação de valores duplicados(de todos os 33 df):
for i in range(qtdTabela):
  brutus[i].duplicated().sum()

In [ ]:
#removendo valores duplicados(de todos os 33 df):
brutus[i].drop_duplicates()

**VALIDAÇÃO SCHEMA**

In [ ]:
#validando o schemma dos dfs
schema = pa.DataFrameSchema(columns = {'Nome':pa.Column(pa.String),
                                       'E-mail':pa.Column(pa.String),
                                       'ID_usuario':pa.Column(pa.String),
                                       'Matriculado_em':pa.Column(pa.DateTime),
                                       'Percentual_completado':pa.Column(pa.Int),
                                       'Numero_de_serie_certificado':pa.Column(pa.String),
                                       'Curso':pa.Column(pa.String)
                                      }
                           )

**SUBSTITUIÇÃO NULOS OU AUSENTES**

In [ ]:
brutus[0].isna().sum()

Nome                            0
E-mail                          0
ID_usuario                      0
Matriculado_em                  0
Percentual_completado           0
Numero_de_serie_certificado    61
Curso                           0
dtype: int64

In [ ]:
#verificando valores nulos ou ausentes(de todos os 33 df):
for i in range(qtdTabela):
  brutus[i].isna().sum()

**COPIA DE DF EM VARIAVEIS** <BR>
tirando o dataframe de uma lista e colocando cada DataFrame em variavel 

In [ ]:
#caso queira um dataframe especifico, foi feito uma validação com o elemento da coluna 
#e criado uma viariavel que recebe o dataframe, logo em seguida é só chamar o df  
for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Administração de Banco de Dados'):
    df_adminbd = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Desenvolvimento Android e Firebase'):
    df_androidfirebase = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Angular'):
    df_angular = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Banco de Dados Relacional'):
    df_bancodedadosrelacional1 = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Banco de Dados Relacional'):
    df_bancodedadosrelacional = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Crie Seu Primeiro App Com Kotlin'):
    df_crieseuprimeiroappkotlin = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Análise de Dados Financeiros Com Python'):
    df_dadosfinanceiroscompython = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Data Studio'):
     df_datastudio = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Desenvolvimento Web com Bootstrap'):
    df_desenvolvimentowebbootstrap = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Desenvolvimento Web Orientado a Objetos'):
    df_desenvolvimentoweborientadoaobjeto = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='DevOps'):
    df_devops = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Git e Github'):
    df_gitgithub = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Google Cloud Platform'):
    df_googlecloud = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Infraestrutura de Dados'):
     df_implementacaobd = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Introdução à Amazon Web Services (AWS)'):
    df_infraestruturadedados = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Introdução à Inteligência Artificial'):
    df_introaws = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Introdução ao Desenvolvimento Web'):
    df_introducaoaia = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Java: Introdução à Linguagem'):
    df_introducaodesenvolvimentoweb = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Java: Orientação a Objetos'):
    df_javaintroducao = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Java Spring Boot'):
    df_javaorientado = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Java Web'):
    df_javaweb = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Kotlin'):
    df_kotlinbasicoaoavancado = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Lógica de Programação'):
    df_logicadeprogramacao = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Metodologias Ágeis'):
    df_metodologiaageis = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Bancos de Dados Não Relacionais (NoSQL)'):
    df_nosql = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Power BI'):
    df_powerbi = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Python e Dados Financeiros'):
    df_pythonfinance2 = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Python: Introdução à Linguagem'):
    df_pythonintroducaoalinguagem = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Machine Learning com Python'):
    df_pythonmachinelearning = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Python: Manipulação de Dados Com Pandas e PySpark'):
    df_pythonmanipulacaodedados = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Python: Orientação a Objetos e Manipulação de Dados'):
    df_pythonorientadoaobjeto = brutus[i].copy()

for i in range(qtdTabela):
  if(brutus[i].loc[0,'Curso']=='Salesforce'):
    df_salesforce = brutus[i].copy()


**DICIONÁRIO DE DATAFRAMES**<br>
Administração de Banco de Dados = df_adminbd,<br>
Desenvolvimento Android e Firebase = df_androidfirebase,<br>
Angular = df_angular,<br>
Banco de Dados Relacional = df_bancodedadosrelacional(1),<br>
Banco de Dados Relacional = df_bancodedadosrelacional,<br>
Crie Seu Primeiro App Com Kotlin = df_crieseuprimeiroappkotlin,<br>
Análise de Dados Financeiros Com Python = df_dadosfinanceiroscompython,<br>
Data Studio = df_datastudio,<br>
Desenvolvimento Web com Bootstrap = df_desenvolvimentowebbootstrap,<br>
Desenvolvimento Web Orientado a Objetos = df_desenvolvimentoweborientadoaobjeto,<br>
DevOps = df_devops,<br>
Git e Github = df_gitgithub,<br>
Google Cloud Platform = df_googlecloud,<br>
Implementação de Banco de Dados = df_implementacaobd,<br>
Infraestrutura de Dados = df_infraestruturadedados,<br>
Introdução à Amazon Web Services (AWS) = df_introaws,<br>
Introdução à Inteligência Artificial = df_introducaoaia,<br>
Introdução ao Desenvolvimento Web = df_introducaodesenvolvimentoweb,<br>
Java: Introdução à Linguagem = df_javaintroducao,<br>
Java: Orientação a Objetos = df_javaorientado,<br>
Java Spring Boot = df_javaspring,<br>
Java Web: Aprenda a Criar Sistemas Web Utilizando Java = df_javaweb,<br>
Kotlin: Básico ao Avançado = df_kotlinbasicoaoavancado,<br>
Lógica de Programação = df_logicadeprogramacao,<br>
Metodologias Ágeis = df_metodologiaageis,<br>
Bancos de Dados Não Relacionais (NoSQL) = df_nosql,<br>
Power BI = df_powerbi,<br>
Python e Dados Financeiros = df_pythonfinance2,<br>
Python: Introdução à Linguagem = df_pythonintroducaoalinguagem,<br>
Machine Learning com Python = df_pythonmachinelearning,<br>
Python: Manipulação de Dados Com Pandas e PySpark = df_pythonmanipulacaodedados,<br>
Python: Orientação a Objetos e Manipulação de Dados = df_pythonorientadoaobjeto,<br>
brutos/salesforce.xlsx = df_salesforce<br>



#USUÁRIOS 

###RECEBIMENTO DE DADOS VIA GCP

**RECEBENDO DADOS**

In [ ]:
#recebendo a tabela "Usuários" separado por ter um tratamento diferente das outras 33
client = storage.Client()

bucket = client.get_bucket('projetofinal_soulcode')

bucket.blob('usuarios.xlsx')

path = 'gs://projetofinal_soulcode/brutos/usuarios.xlsx'

In [ ]:
#lendo o df usuarios
df_usuarios = pd.read_excel(path)

**COPA DO DF ORIGINAL**

In [ ]:
#realizando um backup para enviar como bruto
df_usuarios_brutos = df_usuarios.copy()

In [ ]:
#verificando os tipos das colunas
df_usuarios.dtypes

###TRATAMENTO DE DADOS


**RENOMEANDO COLUNAS**

In [ ]:
#Renomeando colunas trazendo para o PT-BR
df_usuarios.rename(columns={'userid':'ID_Usuario','fullname':'Nome','email':'Email','lastlogin':'Ultimo_login'},inplace=True)
df_usuarios.rename(columns={'joined_at':'Matriculou_em','course_count' :'Contagem_de_cursos','src':'Src','country':'Pais'},inplace=True)
df_usuarios.rename(columns={'sign_in_count':'Contagem_de_login','unsubscribe_from_marketing_emails':'Cancelar_a_assinatura_de_emails_de_marketing','affiliate_code':'Codigo_de_afiliado','billing_address_line':'Linha_de_endereco_de_faturamento'},inplace=True)
df_usuarios.rename(columns={'billing_address_city':'Cidade_do_endereco_de_cobranca','billing_address_region':'Regiao_do_endereco_de_cobranca','billing_address_zipcode' :'CEP_do_endereco_de_cobranca','billing_address_country':'Pais_do_endereco_de_cobranca'},inplace=True)
df_usuarios.rename(columns={'delivery_address_line':'Linha_do_endereco_de_entrega','delivery_address_city':'Cidade_do_endereco_de_entrega'},inplace=True)
df_usuarios.rename(columns={'delivery_address_region':'Regiao_do_endereco_de_entrega','delivery_address_zipcode':'CEP_do_endereco_de_entrega','delivery_address_country':'Pais_do_endereco_de_entrega'},inplace=True)

In [ ]:
df_usuarios.dtypes

**ALTERAÇÃO NA DATA/HORA**

In [ ]:
#mudando o campo de data para deixar padronizado 
df_usuarios['Matriculou_em'] = pd.to_datetime(df_usuarios['Matriculou_em'])
df_usuarios['Matriculou_em'] = df_usuarios['Matriculou_em'].dt.strftime('%d-%m-%Y')


In [ ]:
#mudando o campo de data para deixar padronizado 
df_usuarios['Ultimo_login'] = pd.to_datetime(df_usuarios['Ultimo_login'])
df_usuarios['Ultimo_login'] = df_usuarios['Ultimo_login'].dt.strftime('%d-%m-%Y')

**ALTERAÇÃO NA COLUNA NOME**  

In [ ]:
#mudando o campo nome para deixar padronizado 
df_usuarios['Nome'] = df_usuarios['Nome'].str.split().apply(lambda x: ' '.join([word.title() for word in x]))

**DROP DA COLUNA TAGS**

In [ ]:
#excluindo a coluna "tags" por só ter um unico valor 
df_usuarios.drop(['tags'], axis=1,inplace=True)

**ALTERAÇÃO DO VALOR DA COLUNA**

In [ ]:
#verificando valores unicos da coluna
sorted(pd.unique(df_usuarios['Cancelar_a_assinatura_de_emails_de_marketing']))

[0.0, 1.0, nan]

In [ ]:
#Trocando os valores da coluna "Cancelar_a_assinatura_de_e-mails_de_marketing" de '0','1' para verdadeiro e falso  
filtroverdadeiro = df_usuarios.Cancelar_a_assinatura_de_emails_de_marketing == 1.0
df_usuarios.loc[filtroverdadeiro,'Cancelar_a_assinatura_de_emails_de_marketing'] = 'Verdadeiro'
filtrofalso = df_usuarios.Cancelar_a_assinatura_de_emails_de_marketing == 0.0
df_usuarios.loc[filtrofalso,'Cancelar_a_assinatura_de_emails_de_marketing'] = 'Falso'

**TRATAMENTO DE DADOS NA/NAN**


In [ ]:
df_usuarios.info()

In [ ]:
#verificando valores nulos e ausentes de cada coluna
df_usuarios.isna().sum()


#LEADERBOARD

###RECEBIMENTO VIA GCP

**RECEBIMENTO DADOS**

In [ ]:
#recebendo a tabela "leaderboard" separado por ter um tratamento diferente das outras 33
client = storage.Client()

bucket = client.get_bucket('projetofinal_soulcode')

bucket.blob('leaderboard.xlsx')

path = 'gs://projetofinal_soulcode/brutos/leaderboard.xlsx'

In [ ]:
df_leaderboard = pd.read_excel(path)

In [ ]:
#backup bruto df_leaderboard
df_leaderboard_bruto = df_leaderboard.copy()

###PRÉ ANÁLISE

In [ ]:
df_leaderboard

###TRATAMENTO DE DADOS

**RENOMEANDO COLUNAS**

In [ ]:
#traduzindo todos os nomes de colunas, de df_leaderboard, para renomear para o PT-BR 
df_leaderboard.rename(columns={'ID_usuário': 'ID_usuario',
                   'Número_de_série_certificado': 'Numero_de_serie_certificado',
                   'name': 'Nome',
                   'email': 'E-mail',
                   'user_id': 'ID_usuario',
                   'enrollments_created_at': 'Matriculado_em',
                   'percent_complete': 'Percentual_completado',
                   'certificate_serial_number': 'Numero_de_serie_certificado'
                   },
                   inplace = True)

**ALTERAÇÃO NA DATA/HORA**

In [ ]:
# Converter a coluna de datas para o formato de data/hora se já não tiver (de df_leaderboard)

df_leaderboard['Matriculado_em'] = pd.to_datetime(df_leaderboard['Matriculado_em'])

df_leaderboard['Matriculado_em'] = df_leaderboard['Matriculado_em'].dt.strftime('%d-%m-%Y %H:%M')


**ALTERAÇÃO NA NOLUNA NOME**

In [ ]:
# Para colocar a primeira letra de cada nome em letra maiúscula(de df_leaderboard): 

df_leaderboard['Nome'] = df_leaderboard['Nome'].str.split().apply(lambda x: ' '.join([word.title() for word in x]))

**VALORES DUPLICADOS**

In [ ]:
# Verificação de valores duplicados(de df_leaderboard):
df_leaderboard.duplicated().sum()

0

In [ ]:
#removendo valores duplicados(de df_leaderboard):
df_leaderboard.drop_duplicates()

**SCHEMA**

In [ ]:
#validando o schema de df_leaderboard
schema = pa.DataFrameSchema(columns = {'Nome':pa.Column(pa.String),
                                       'E-mail':pa.Column(pa.String),
                                       'ID_usuario':pa.Column(pa.String),
                                       'Matriculado_em':pa.Column(pa.DateTime),
                                       'Percentual_completado':pa.Column(pa.Int),
                                       'Numero_de_serie_certificado':pa.Column(pa.String),
                                       'Curso':pa.Column(pa.String)
                                      }
                           )

**VALORES NA/NAN**

In [ ]:
# Substituindo os valores nulos(de df_leaderboard):
# df_leaderboard = df_leaderboard.fillna({'Numero_de_serie_certificado': '-'})

In [ ]:
#verificando valores nulos ou ausentes(de df_leaderboard):
df_leaderboard.isna().sum()

#TABELÃO 

In [ ]:
#concatenando a todas as colundas dos 33 dfs para um df só.  
df_tabelao = pd.concat(brutus)

#IMPORTAÇÕES

##.. 

**MUDANDO LISTA "nomeDosDf"**

In [ ]:
#entrando na lista, acessando cada elemento de string, retirando o 'brutos/' e o '.xlsx'
for i in range(qtdTabela):
  if(nomeDosDf[i][0:6] in 'brutos/'): 
    n = nomeDosDf[i][7:-5]
    nomeDosDf[i] = n 
  else:(print ('deu errado'))

In [ ]:
#verificando os valores recebidos acima
nomeDosDf

#ENVIANDO PARA O BUCKET

###TRATADOS

**CURSOS**

In [ ]:
#enviando os 33 dataframes relacionados a cursos tratados para a pasta "tratados_automocao_xlsx" do bucket(em forma de excel)
for i in range(qtdTabela):
  brutus[i].to_excel(f'gs://projetofinal_soulcode/tratados_automocao_xlsx/{nomeDosDf[i]}.xlsx', index=False)

In [ ]:
#enviando os 33 dataframes relacionados a cursos tratados para a pasta "tratados_automocao_csv" do bucket(em forma de csv)
for i in range(qtdTabela):
  brutus[i].to_csv(f'gs://projetofinal_soulcode/tratados_automocao_csv/{nomeDosDf[i]}.csv', index=False)

In [ ]:
#salvando o arquivo excel no pc 
df_tabelao.to_excel(f'df_tabelao.xlsx', index=False)

**USUÁRIOS**

In [ ]:
#enviando o dataframe "Usuário" tratado em, respectivamente, excel e csv
df_usuarios.to_excel(f'gs://projetofinal_soulcode/tratados_automocao_xlsx/usuarios_tratados.xlsx', index=False)
df_usuarios.to_csv(f'gs://projetofinal_soulcode/tratados_automocao_csv/usuarios_tratados.csv', index=False)   

**LEADERBOARD**

In [ ]:
#enviando o dataframe "leaderboard" tratado em, respectivamente, excel e csv
df_leaderboard.to_excel(f'gs://projetofinal_soulcode/tratados_automocao_xlsx/df_leaderboard.xlsx', index=False)
df_leaderboard.to_csv(f'gs://projetofinal_soulcode/tratados_automocao_csv/df_leaderboard.csv', index=False)   


**TABELAO**

In [ ]:
#enviando o dataframe "tabelao" tratado em excel 
df_tabelao.to_excel(f'gs://projetofinal_soulcode/tratados_automocao_xlsx/df_tabelao.xlsx', index=False)


In [ ]:
#transformando o aquivo do excel para csv 
df_emcsv = pd.DataFrame(pd.read_excel("df_tabelao.xlsx"))

In [ ]:
#enviando o dataframe "tabelao" tratado em csv
df_emcsv.to_csv(f'gs://projetofinal_soulcode/tratados_automocao_csv/tabelaonovo.csv', index=False) 

###BRUTOS

**CURSOS**

In [ ]:
#enviando os 33 dataframes relacionados a cursos brutos para a pasta "arquivos_originais_excel" do bucket(em forma de excel)
for i in range(qtdTabela):
  dados_cursos_bruto[i].to_excel(f'gs://projetofinal_soulcode/arquivos_originais_excel/{nomeDosDf[i]}.xlsx', index=False)


In [ ]:
#enviando os 33 dataframes relacionados a cursos brutos para a pasta "brutos_automocao_csv" do bucket(em forma de csv)
for i in range(qtdTabela):
  dados_cursos_bruto[i].to_csv(f'gs://projetofinal_soulcode/brutos_automocao_csv/{nomeDosDf[i]}.csv', index=False)


**USUÁRIOS**

In [ ]:
#enviando o dataframe "Usuário" bruto em, respectivamente, excel e csv
df_usuarios_brutos.to_excel(f'gs://projetofinal_soulcode/arquivos_originais_excel/df_usuarios_brutos.xlsx', index=False)
df_usuarios_brutos.to_csv(f'gs://projetofinal_soulcode/brutos_automocao_csv/df_usuarios_brutos.csv', index=False)

**LEADERBOARD**

In [ ]:
#enviando o dataframe "leaderboard" bruto em, respectivamente, excel e csv
df_leaderboard_bruto.to_excel(f'gs://projetofinal_soulcode/arquivos_originais_excel/df_leaderboard_bruto.xlsx', index=False)
df_leaderboard_bruto.to_csv(f'gs://projetofinal_soulcode/brutos_automocao_csv/df_leaderboard_bruto.csv', index=False)

#SALVANDO NO COMPUTADOR 

###TRATADOS

**CURSOS**

In [ ]:
for i in range(qtdTabela):
  brutus[i].to_csv(f'{nomeDosDf[i]}.csv', index=False)

In [ ]:
for i in range(qtdTabela):
  brutus[i].to_excel(f'{nomeDosDf[i]}.xlsx', index=False)

**USUÁRIOS**

In [ ]:
df_usuarios.to_excel(f'df_usuarios.xlsx', index=False)
df_usuarios.to_csv(f'df_usuarios.csv', index=False)   

**LEADERBOARD**

In [ ]:
df_leaderboard.to_excel(f'df_leaderboard.xlsx', index=False)
df_leaderboard.to_csv(f'df_leaderboard.csv', index=False)   

**TABELAO**

In [ ]:
df_tabelao.to_excel(f'df_tabelao.xlsx', index=False)
df_tabelao.to_csv(f'df_tabelao.csv', index=False)   

###BRUTOS

**CURSOS**

In [ ]:
for i in range(qtdTabela):
  brutus[i].to_csv(f'{nomeDosDf[i]}.csv', index=False)

In [ ]:
for i in range(qtdTabela):
  brutus[i].to_excel(f'{nomeDosDf[i]}.xlsx', index=False)

**USUARIOS**

In [ ]:
df_usuarios_brutos.to_excel(f'usuarios_brutos.xlsx', index=False)
df_usuarios_brutos.to_csv(f'usuarios_brutos.csv', index=False)   

**LEADERBOARD**

In [ ]:
df_leaderboard_bruto.to_excel(f'df_leaderboard_bruto.xlsx', index=False)
df_leaderboard_bruto.to_csv(f'df_leaderboard_bruto.csv', index=False)  

#ENVIAR DIRETO PARA O BIGQUERY


###TRATADOS

In [ ]:
client = bigquery.Client()

**CURSOS**

In [ ]:
#Enviando dataset dos cursos 
dataset_id = 'projetofinaltratados'
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'
client.create_dataset('tabelas_cursos')

In [ ]:
for i in range (qtdTabela):
  table_name = 'tabelas_cursos'
  table_ref = dataset_ref.table(table_name)
  job_config = bigquery.LoadJobConfig()
  job_config.autodetect = True
  job_config.source_format = bigquery.SourceFormat.CSV
  job_config.skip_leading_rows = 1
  source_uri = f'gs://projetofinal_soulcode/tratados_automocao_csv/{nomeDosDf[i]}.csv'

  job = client.load_table_from_uri(
          source_uri,
          table_ref,
          location='US',
          job_config=job_config)  
  job.result()

**USUÁRIO**

In [ ]:
#Enviando dataset de usuários
dataset_id = 'projetofinaltratados'
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'
client.create_dataset('usuarios')

In [ ]:
table_name = 'usuarios'
table_ref = dataset_ref.table(table_name)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
source_uri = 'gs://projetofinal_soulcode/tratados_automocao_csv/usuarios_tratados.csv'

job = client.load_table_from_uri(
        source_uri,
        table_ref,
        location='US',
        job_config=job_config)  
job.result()

**LEADERBOARD**

In [ ]:
#Enviando dataset de usuários
dataset_id = 'projetofinaltratados'
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'
client.create_dataset('leaderboard')

In [ ]:
table_name = 'leaderboard'
table_ref = dataset_ref.table(table_name)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
source_uri = 'gs://projetofinal_soulcode/tratados_automocao_csv/df_leaderboard.csv'

job = client.load_table_from_uri(
        source_uri,
        table_ref,
        location='US',
        job_config=job_config)  
job.result()

**TABELAO**

In [ ]:
dataset_id = 'projetofinaltratados'
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'
client.create_dataset('tabelao')

In [ ]:
table_name = 'tabelao'
table_ref = dataset_ref.table(table_name)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
source_uri = 'gs://projetofinal_soulcode/tratados_automocao_csv/df_tabelao.csv'

job = client.load_table_from_uri(
        source_uri,
        table_ref,
        location='US',
        job_config=job_config)  
job.result()

#ENVIANDO PARA O MONGODB

###TRATADOS

**CONECTOR MONGODB**

In [ ]:
uri = "mongodb+srv://clusteraula0.on8lj7t.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client2 = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='/content/X509-cert-854953342432239118.pem',
                     )

**CURSOS**

In [ ]:
for i in range(qtdTabela):

  db = client2['Tratados_ProjetoFinal']
  colecao = db[f'{nomeDosDf[i]}.xls']
   
  df_dict = brutus[i].to_dict("records")

  colecao.insert_many(df_dict)

**USUÁRIOS** 

In [ ]:
db1 = client2['Tratados_ProjetoFinal']
colecao = db1['usuarios.xls']
   
df_dict = df_usuarios.to_dict("records")

colecao.insert_many(df_dict)

**LEADERBOARD**

In [ ]:
db2 = client2['Tratados_ProjetoFinal']
colecao = db2['leaderboard.xls']
   
df_dict = df_leaderboard.to_dict("records")

colecao.insert_many(df_dict)

**TABELAO**

In [ ]:
db3 = client2['Tratados_ProjetoFinal']
colecao = db3['tabelao.xls']
   
df_dict = df_tabelao.to_dict("records")

colecao.insert_many(df_dict)

###BRUTOS

**CURSOS**

In [ ]:
for i in range(qtdTabela):

  db4 = client2['Brutos_ProjetoFinal']
  colecao = db4[f'{nomeDosDf[i]}.xls']
   
  df_dict = dados_cursos_bruto[i].to_dict("records")

  colecao.insert_many(df_dict)

**USUÁRIOS** 

In [ ]:
db5 = client2['Brutos_ProjetoFinal']
colecaoproj = db5['Usuário_bruto_final']
   
df_dict = df_usuarios_brutos.to_dict("records")

colecao.insert_many(df_dict)

**LEADERBOARD**

In [ ]:
db6 = client2['Brutos_ProjetoFinal']
colecao = db6['leaderboard.xls']
   
df_dict = df_leaderboard_bruto.to_dict("records")

colecao.insert_many(df_dict)

#POSSIVEIS INCONSISTÊNCIAS 

In [ ]:
df_usuarios_insights = df_usuarios.copy()

##USUÁRIOS GRAN E SOULCODE

In [ ]:
filtrogran = df_usuarios_insights.Email.str[-23:] == 'grancursosonline.com.br'
df_filtrogran = df_usuarios_insights.loc[filtrogran] 

In [ ]:
df_filtrogran

In [ ]:
filtrosoulcode = df_usuarios_insights.Email.str[-12:] == 'soulcode.com'
df_filtrosoulcode = df_usuarios_insights.loc[filtrosoulcode] 

##35 CURSOS

In [ ]:
filtro_35 = df_usuarios_insights.Contagem_de_cursos == 35
df_filtro_35 = df_usuarios_insights.loc[filtro_35] 


##34 CURSOS

In [ ]:
filtro_34 = df_usuarios_insights.Contagem_de_cursos == 34
df_filtro_34 = df_usuarios_insights.loc[filtro_34] 

In [ ]:
df_usu_estranhos = pd.concat([df_filtro_34,df_filtro_35,df_filtrosoulcode,df_filtrogran]) 

##SUBINDO PRO BQ

In [ ]:
df_usu_estranhos.to_excel('gs://projetofinal_soulcode/tratados_automocao_xlsx/df_usu_estranhos.xlsx', index=False)
df_usu_estranhos.to_csv('gs://projetofinal_soulcode/tratados_automocao_csv/df_usu_estranhos.csv', index=False)   

In [ ]:
client = bigquery.Client()

In [ ]:
#Enviando dataset dos cursos 
dataset_id = 'projetofinaltratados'
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = 'US'
client.create_dataset('usuarios_estranhos')

In [ ]:
table_name = 'usuarios_estranhos1'
table_ref = dataset_ref.table(table_name)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
source_uri = 'gs://projetofinal_soulcode/tratados_automocao_csv/df_usu_estranhos.csv'

job = client.load_table_from_uri(
        source_uri,
        table_ref,
        location='US',
        job_config=job_config)  
job.result()